In [1]:

from sklearn.tree import DecisionTreeClassifier
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score
import pandas as pd
from pyswarms.single.global_best import GlobalBestPSO
from sklearn.naive_bayes import GaussianNB
import joblib

In [2]:
data = pd.read_csv('dataset_1.csv')

figure_dict = {
    'circle': 0,
    'rectangle': 1
}
data['figure1'] = data['figure1'].map(figure_dict)
data['figure2'] = data['figure2'].map(figure_dict)


X = data.drop('collision', axis=1)
y = data['collision']
# Предобработка
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

In [3]:
# Определение диапазонов гиперпараметров
decision_tree_param_ranges = {
    'max_depth': (1, 20),  # Глубина дерева
    'min_samples_split': (2, 20),  # Минимальное количество выборок для разделения
    'min_samples_leaf': (1, 20)  # Минимальное количество выборок в листе
}

naive_bayes_param_ranges = {
    'var_smoothing': (1e-11, 1e-7)
}

In [4]:
# Целевая функция для Decision Tree
def decision_tree_fitness_function(params):
    # Преобразуем 2D-массив в 1D (если передаётся несколько частиц)
    if params.ndim == 2:
        params = params[0]  # Берём первую частицу (можно цикл, если нужно для всех)
    
    max_depth = int(params[0])
    min_samples_split = int(params[1])
    min_samples_leaf = int(params[2])
    
    model = DecisionTreeClassifier(
        max_depth=max_depth,
        min_samples_split=min_samples_split,
        min_samples_leaf=min_samples_leaf,
        random_state=42
    )
    
    model.fit(X_train, y_train)
    y_pred = model.predict(X_test)
    accuracy = accuracy_score(y_test, y_pred)
    return -accuracy  # PSO минимизирует функцию, поэтому возвращаем -accuracy

In [5]:
# Целевая функция для Logistic Regression
def naive_bayes_fitness_function(params):
    if params.ndim == 2:
        params = params[0]
    
    var_smoothing = float(params[0])
    
    model = GaussianNB(
        var_smoothing=var_smoothing
    )
    
    model.fit(X_train, y_train)
    y_pred = model.predict(X_test)
    accuracy = accuracy_score(y_test, y_pred)
    return -accuracy


In [6]:
# Настройка PSO для Decision Tree
bounds_dt = (
    [
        decision_tree_param_ranges['max_depth'][0],           # Нижняя граница max_depth
        decision_tree_param_ranges['min_samples_split'][0],   # Нижняя граница min_samples_split
        decision_tree_param_ranges['min_samples_leaf'][0]     # Нижняя граница min_samples_leaf
    ],
    [
        decision_tree_param_ranges['max_depth'][1],           # Верхняя граница max_depth
        decision_tree_param_ranges['min_samples_split'][1],   # Верхняя граница min_samples_split
        decision_tree_param_ranges['min_samples_leaf'][1]     # Верхняя граница min_samples_leaf
    ]
)

optimizer_dt = GlobalBestPSO(
    n_particles=10,  # Количество частиц
    dimensions=3,  # Количество гиперпараметров
    options={'c1': 0.5, 'c2': 0.3, 'w': 0.9},  # Параметры PSO
    bounds=bounds_dt
)

In [7]:
print("Дерево решений")
best_cost_dt, best_pos_dt = optimizer_dt.optimize(decision_tree_fitness_function, iters=20)
print(f"Лучшие параметры: max_depth={int(best_pos_dt[0])}, min_samples_split={int(best_pos_dt[1])}, min_samples_leaf={int(best_pos_dt[2])}")
print(f"Лучший Accuracy: {-best_cost_dt}")

best_dt_model = DecisionTreeClassifier(
    max_depth=int(best_pos_dt[0]),
    min_samples_split=int(best_pos_dt[1]),
    min_samples_leaf=int(best_pos_dt[2]),
    random_state=42
)
joblib.dump(best_dt_model, '2_best_decision_tree_model.pkl')

2025-04-19 00:25:19,690 - pyswarms.single.global_best - INFO - Optimize for 20 iters with {'c1': 0.5, 'c2': 0.3, 'w': 0.9}


Дерево решений


pyswarms.single.global_best: 100%|██████████|20/20, best_cost=-0.5
2025-04-19 00:25:19,767 - pyswarms.single.global_best - INFO - Optimization finished | best cost: -0.5, best pos: [13.33134552 16.50984883 13.40098912]


Лучшие параметры: max_depth=13, min_samples_split=16, min_samples_leaf=13
Лучший Accuracy: 0.5


['2_best_decision_tree_model.pkl']

In [8]:
bounds_nb = (
    [naive_bayes_param_ranges['var_smoothing'][0]],  # Нижняя граница var_smoothing
    [naive_bayes_param_ranges['var_smoothing'][1]]   # Верхняя граница var_smoothing
)

optimizer_nb = GlobalBestPSO(
    n_particles=10,
    dimensions=1,  # Только один параметр для оптимизации
    options={'c1': 0.5, 'c2': 0.3, 'w': 0.9},
    bounds=bounds_nb
)

In [9]:
print("\nНаивный Байес")
best_cost_lr, best_pos_lr = optimizer_nb.optimize(naive_bayes_fitness_function, iters=20)
print(f"Лучший параметр: {best_pos_lr[0]}")
print(f"Лучший Accuracy: {-best_cost_lr}")

best_nb_model = GaussianNB(
    var_smoothing=float(best_pos_lr[0])
)
joblib.dump(best_nb_model, '2_best_naive_bayes_model.pkl')

2025-04-19 00:25:19,783 - pyswarms.single.global_best - INFO - Optimize for 20 iters with {'c1': 0.5, 'c2': 0.3, 'w': 0.9}



Наивный Байес


pyswarms.single.global_best: 100%|██████████|20/20, best_cost=-0.8
2025-04-19 00:25:19,835 - pyswarms.single.global_best - INFO - Optimization finished | best cost: -0.8, best pos: [4.43184704e-08]


Лучший параметр: 4.4318470416031076e-08
Лучший Accuracy: 0.8


['2_best_naive_bayes_model.pkl']